In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1701298805849_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
spark = SparkSession.builder.appName("YelpAnalysis").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
s3_path = "s3://myyelpdataset/Outputs/Reviews/Parquet/"

# Read the Parquet file into a DataFrame
df = spark.read.parquet(s3_path)
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+--------------------+-------------------+
|           review_id|         business_id|stars|                text|               date|
+--------------------+--------------------+-----+--------------------+-------------------+
|6AUbwLYs9RJG_mqvd...|OJEiB_a2I_gxC-Zea...|  3.0|The food was good...|2020-01-16 17:37:40|
|01YL43edpWZ1oOBp-...|B0WkL720J0N6PjQPm...|  5.0|My son has been g...|2020-01-22 03:16:59|
|FoIQOYlkg1cycFit8...|Ct_g_xmNh1wvpfTD-...|  5.0|Good place for ph...|2020-03-14 14:03:10|
|V7Qoi8KjZSE_k_-GS...|3qpubK1C8SsDcq8Be...|  5.0|My boyfriend and ...|2018-08-16 23:55:31|
|fXTW03d-GaZrkYj4D...|gXiWHYBuPLWWXZJPO...|  5.0|No complaints alt...|2019-11-09 21:42:15|
|Al__qgTzik2s4H76B...|diESSke07mVElwq0p...|  5.0|Dr. DeBruin is a ...|2019-03-07 23:14:19|
|YPC1kwi8wZ_BhkYv7...|v3OZ36tMFyAuQiqaY...|  1.0|The man who took ...|2012-05-31 15:09:23|
|ngK7--8PaU5QQJvyS...|Q2VFD1n1THRsSnAD_...|  1.0|I wouldn't even g...|2019-12-05 02:17:14|

In [ ]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- review_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- date: string (nullable = true)

In [ ]:
sc.install_pypi_package("textblob")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from pyspark.sql.functions import year, month, dayofmonth, to_date

# Assuming 'date' is a string column
df = df.withColumn('date', to_date(col('date'), 'yyyy-MM-dd HH:mm:ss'))

# Extracting components
df = df.withColumn('year', year(df['date']))
df = df.withColumn('month', month(df['date']))
df = df.withColumn('day', dayofmonth(df['date']))

# Show results
df.select("business_id", "text", "year", "month", "day").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+----+-----+---+
|         business_id|                text|year|month|day|
+--------------------+--------------------+----+-----+---+
|OJEiB_a2I_gxC-Zea...|The food was good...|2020|    1| 16|
|B0WkL720J0N6PjQPm...|My son has been g...|2020|    1| 22|
|Ct_g_xmNh1wvpfTD-...|Good place for ph...|2020|    3| 14|
|3qpubK1C8SsDcq8Be...|My boyfriend and ...|2018|    8| 16|
|gXiWHYBuPLWWXZJPO...|No complaints alt...|2019|   11|  9|
+--------------------+--------------------+----+-----+---+
only showing top 5 rows

This is identified Top restaurant with best Average rating - Ruby Slipper - New Orleans

In [ ]:
df_top = df.filter((df['business_id'] == 'oBNrLz4EDhiscSlbOl8uAw'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_top = df_top.drop('review_id', 'date', 'month', 'day')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_top.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+--------------------+----+
|         business_id|stars|                text|year|
+--------------------+-----+--------------------+----+
|oBNrLz4EDhiscSlbO...|  4.0|Went in for break...|2017|
|oBNrLz4EDhiscSlbO...|  5.0|I'm glad we tried...|2018|
|oBNrLz4EDhiscSlbO...|  2.0|My boyfriend had ...|2015|
|oBNrLz4EDhiscSlbO...|  4.0|The Ruby Slipper ...|2015|
|oBNrLz4EDhiscSlbO...|  1.0|This place has th...|2018|
+--------------------+-----+--------------------+----+
only showing top 5 rows

In [ ]:
# Check the distribution of stars for different years
df_top.groupBy('year').agg({'stars': 'mean'}).orderBy('year').show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------------+
|year|       avg(stars)|
+----+-----------------+
|2010|            4.375|
|2011|3.878787878787879|
|2012|              4.0|
|2013|3.865168539325843|
|2014|4.140672782874618|
|2015|4.231388329979879|
|2016|4.258652094717669|
|2017|4.327844311377246|
|2018|4.253968253968254|
|2019|4.394444444444445|
|2020|4.522184300341297|
|2021|4.419062027231467|
|2022|4.703703703703703|
+----+-----------------+

In [ ]:
from pyspark.sql import functions as F

# Group by 'year' and count the reviews
count_by_year = df_top.groupBy('year').agg(F.count('text').alias('review_count'))
count_by_year_sorted = count_by_year.orderBy('year')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
count_by_year_sorted.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------------+
|year|review_count|
+----+------------+
|2010|           8|
|2011|          99|
|2012|         112|
|2013|         178|
|2014|         327|
|2015|         497|
|2016|         549|
|2017|         668|
|2018|         945|
|2019|         900|
|2020|         293|
|2021|         661|
|2022|          27|
+----+------------+

In [ ]:
df_top.write.parquet("s3://myyelpdataset/EMR_Sentiment_Analysis/top/allyear/", mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Filter for years after 2020
df_top_filtered= df_top.filter(df_top['year'] > 2020)
df_top_filtered.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+--------------------+----+
|         business_id|stars|                text|year|
+--------------------+-----+--------------------+----+
|oBNrLz4EDhiscSlbO...|  1.0|The waitress is v...|2021|
|oBNrLz4EDhiscSlbO...|  5.0|Visiting Ruby Sli...|2021|
|oBNrLz4EDhiscSlbO...|  5.0|Always delicious ...|2021|
|oBNrLz4EDhiscSlbO...|  1.0|Im\nSure its grea...|2021|
|oBNrLz4EDhiscSlbO...|  5.0|Nicole at the bar...|2021|
+--------------------+-----+--------------------+----+
only showing top 5 rows

In [ ]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import DoubleType, StringType
from textblob import TextBlob

# Function for sentiment analysis using TextBlob
def detect_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

# Register UDF
sentiment_udf = udf(detect_sentiment, DoubleType())

# Calculate sentiment score
def sentiment_score(polarity):
    if polarity > 0:
        return 'POSITIVE'
    elif polarity < 0:
        return 'NEGATIVE'
    else:
        return 'NEUTRAL'

score_udf = udf(sentiment_score, StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Apply sentiment analysis using TextBlob
df_with_sentiment = df_top_filtered.withColumn("polarity", sentiment_udf(col("text")))
df_final = df_with_sentiment.withColumn("sentiment", score_udf(col("polarity")))

# Show results
df_final.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+--------------------+----+------------------+---------+
|         business_id|stars|                text|year|          polarity|sentiment|
+--------------------+-----+--------------------+----+------------------+---------+
|oBNrLz4EDhiscSlbO...|  1.0|The waitress is v...|2021|0.0688888888888889| POSITIVE|
|oBNrLz4EDhiscSlbO...|  5.0|Visiting Ruby Sli...|2021|0.5928571428571429| POSITIVE|
|oBNrLz4EDhiscSlbO...|  5.0|Always delicious ...|2021|0.5116666666666666| POSITIVE|
|oBNrLz4EDhiscSlbO...|  1.0|Im\nSure its grea...|2021|               0.6| POSITIVE|
|oBNrLz4EDhiscSlbO...|  5.0|Nicole at the bar...|2021|               0.8| POSITIVE|
+--------------------+-----+--------------------+----+------------------+---------+
only showing top 5 rows

In [ ]:
df_final.write.option("header", "true").mode("overwrite").csv("s3://myyelpdataset/EMR_Sentiment_Analysis/top/prqt/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_final.write.parquet("s3://myyelpdataset/EMR_Sentiment_Analysis/top/prqt/", mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

This is identified as restaurant with can be improved w.r.t Average rating - Burbon House - New Orleans

In [ ]:
df_sug = df.filter((df['business_id'] == 'iwmW2mgcn2YdirXUHCsgXQ'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_sug = df_sug.drop('review_id', 'date', 'month', 'day')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_sug.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+--------------------+----+
|         business_id|stars|                text|year|
+--------------------+-----+--------------------+----+
|iwmW2mgcn2YdirXUH...|  2.0|Came here for the...|2017|
|iwmW2mgcn2YdirXUH...|  2.0|We ordered raw an...|2016|
|iwmW2mgcn2YdirXUH...|  5.0|The drinks are we...|2018|
|iwmW2mgcn2YdirXUH...|  4.0|I will never forg...|2017|
|iwmW2mgcn2YdirXUH...|  4.0|Wonderful atmosph...|2017|
+--------------------+-----+--------------------+----+
only showing top 5 rows

In [ ]:
# Check the distribution of stars for different years
df_sug.groupBy('year').agg({'stars': 'mean'}).orderBy('year').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------------------+
|year|        avg(stars)|
+----+------------------+
|2006|               4.0|
|2007| 4.142857142857143|
|2008|3.6470588235294117|
|2009|3.7096774193548385|
|2010|3.5510204081632653|
|2011|3.8795180722891565|
|2012|3.3628318584070795|
|2013|3.2783505154639174|
|2014|               3.3|
|2015|3.3783783783783785|
|2016|3.6991150442477876|
|2017|3.6074074074074076|
|2018|3.8481848184818483|
|2019| 3.550387596899225|
|2020| 3.475609756097561|
|2021| 3.645390070921986|
|2022|               5.0|
+----+------------------+

In [ ]:
# Group by 'year' and count the reviews
count_by_year = df_sug.groupBy('year').agg(F.count('text').alias('review_count'))
count_by_year_sorted = count_by_year.orderBy('year')
count_by_year_sorted.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------------+
|year|review_count|
+----+------------+
|2006|           1|
|2007|           7|
|2008|          17|
|2009|          31|
|2010|          49|
|2011|          83|
|2012|         113|
|2013|          97|
|2014|         160|
|2015|         222|
|2016|         226|
|2017|         270|
|2018|         303|
|2019|         258|
|2020|          82|
|2021|         141|
|2022|           4|
+----+------------+

In [ ]:
df_sug.write.parquet("s3://myyelpdataset/EMR_Sentiment_Analysis/sugg/allyear/", mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Filter for years after 2020
df_sug_filtered= df_sug.filter(df_top['year'] > 2020)
df_sug_filtered.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+--------------------+----+
|         business_id|stars|                text|year|
+--------------------+-----+--------------------+----+
|iwmW2mgcn2YdirXUH...|  5.0|The charbroiled o...|2021|
|iwmW2mgcn2YdirXUH...|  5.0|Food was awesome ...|2021|
|iwmW2mgcn2YdirXUH...|  2.0|We called for a r...|2021|
|iwmW2mgcn2YdirXUH...|  5.0|Very good bourbon...|2021|
|iwmW2mgcn2YdirXUH...|  1.0|Decided to have l...|2021|
+--------------------+-----+--------------------+----+
only showing top 5 rows

In [ ]:
# Apply sentiment analysis using TextBlob
df_with_sentiment = df_sug_filtered.withColumn("polarity", sentiment_udf(col("text")))
df_final = df_with_sentiment.withColumn("sentiment", score_udf(col("polarity")))

# Show results
df_final.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+--------------------+----+-------------------+---------+
|         business_id|stars|                text|year|           polarity|sentiment|
+--------------------+-----+--------------------+----+-------------------+---------+
|iwmW2mgcn2YdirXUH...|  5.0|The charbroiled o...|2021|             0.1875| POSITIVE|
|iwmW2mgcn2YdirXUH...|  5.0|Food was awesome ...|2021|0.37463474025974025| POSITIVE|
|iwmW2mgcn2YdirXUH...|  2.0|We called for a r...|2021|0.39285714285714296| POSITIVE|
|iwmW2mgcn2YdirXUH...|  5.0|Very good bourbon...|2021| 0.6616666666666666| POSITIVE|
|iwmW2mgcn2YdirXUH...|  1.0|Decided to have l...|2021|0.02023809523809524| POSITIVE|
+--------------------+-----+--------------------+----+-------------------+---------+
only showing top 5 rows

In [ ]:
df_final.write.option("header", "true").mode("overwrite").csv("s3://myyelpdataset/EMR_Sentiment_Analysis/sugg/prqt/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_final.write.parquet("s3://myyelpdataset/EMR_Sentiment_Analysis/sugg/prqt/", mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…